# GraphQL from Trello

This notebook tests pulling data from Trello via GraphQL query. 

It helps to bring back all the lists and cards on the kanban board. 

In [ ]:
import requests
import json
import pandas as pd
import os
import base64


Pull in all variables - these are stored as secrets and defined either in the project or passed in to codespaces when it is started. 

In [ ]:
trello_username = os.getenv('ATLASSIAN_USERNAME')
trello_graphql_token = os.getenv('ATLASSIAN_API_TOKEN')
trello_repo_name = os.getenv('ATLASSIAN_SUBDOMAIN')
trello_board_id = os.getenv('TRELLO_BOARD_ID')

The GraphQL query to sent to the Atlassian endpoint. This pulls all cards grouped by each list from a specific board. 

In [ ]:
query = """
            query MyQuery {
              trello @optIn(to: ["TrelloBoard", "TrelloListCards"]){
                board(
                  id: "ari:cloud:trello::board/workspace/6137a3a268cbbd3f699e54fc/5f0d82321591070cf48987d5"
                ) {
                  name
                  lists {
                    nodes {
                      name
                      id
                      cards {
                        edges {
                          node {
                            id
                            name
                            labels {
                              nodes {
                                name
                              }
                            }
                          }
                        }
                      }
                    }
                  }
                }
              }
            }
        """

Generate authentication token for basic authentication. 

See [Atlassian Basic Auth instructions](https://developer.atlassian.com/cloud/jira/platform/basic-auth-for-rest-apis/) to learn more

In [ ]:
auth_plain = trello_username + ":" + trello_graphql_token
message_bytes = auth_plain.encode('ascii')
auth_base64 = base64.b64encode(message_bytes)
base64_message = auth_base64.decode('ascii')

my_api_token = 'Basic ' + base64_message

Run the query against the Atlassian GraphQL endpoint

In [ ]:
url = f"https://{trello_repo_name}.atlassian.net/gateway/api/graphql"
headers = {"Authorization": my_api_token}
r = requests.post(url, headers=headers, json={'query': query})
print(r.status_code)

Extract the results in json

In [ ]:
json_data = json.loads(r.text)

Flatten (normalise) the nested json to the bits that matter:

- The cards
- The list they're part of
- The label (if exists)

Labels need to be cleaned up some more.

In [ ]:
storage_json = json_data['data']['trello']['board']['lists']['nodes']
df = pd.json_normalize(storage_json, ["cards","edges"],["name","id"])
df.rename(columns={"node.id":"Card_ID", "node.name":"Card_name","node.labels.nodes":"Card_labels","name":"List_name","id":"List_ID"})


Troubleshooting: Dump output of json into a pretty nested format. 

In [ ]:
print(json.dumps(json_data, indent = 2, sort_keys=True))

Troubleshooting: Extract out a subset into a pretty nested format. 

In [ ]:
print(json.dumps(storage['data']['trello']['board']['lists']['nodes'], indent = 2, sort_keys=True))